In [ ]:
!pip install accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 15.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# load the required packages.

import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [ ]:
import pandas as pd
from datasets import Dataset

# Load the CSV file
train_df = pd.read_csv("training_ROCStories.csv")
# Select only the required columns
train_df = train_df[['storytitle', 'sentence']]

test_df = pd.read_csv("testing_ROCStories.csv")
# Select only the required columns
test_df = test_df[['storytitle', 'sentence']]
# Convert pandas DataFrame to Hugging Face Dataset format

data = Dataset.from_pandas(train_df)

print(data)

Dataset({
    features: ['storytitle', 'sentence'],
    num_rows: 8819
})


In [ ]:
data[0]

{'storytitle': 'Lucky Tickets',
 'sentence': "Quinn couldn't believe her luck. She had won free concert tickets to her favorite music artist. Quinn wanted to invite both her best friends. But she could only pick one. Quinn decided to just give both tickets to her friends."}

In [ ]:
# from sklearn.model_selection import train_test_split
# train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

In [ ]:
#dataset="burkelibbey/colors"
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model="output_model"

In [ ]:
# we need to reformat the data in teh ChatML format.

def formatted_train(input,response)->str:
    return f"<|im_start|>user\n{input}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"

In [ ]:
data = data.to_pandas()
data["text"] = data[["storytitle", "sentence"]].apply(lambda x: "<|im_start|>user\n" + x["storytitle"] + " <|im_end|>\n<|im_start|>assistant\n" + x["sentence"] + "<|im_end|>\n", axis=1)

In [ ]:
data = Dataset.from_pandas(data)

data


Dataset({
    features: ['storytitle', 'sentence', 'text'],
    num_rows: 8819
})

In [ ]:
def get_model_and_tokenizer(mode_id):

    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [ ]:
!pip install -i https://test.pypi.org/simple/bitsandbytes

ERROR: You must give at least one requirement to install (see "pip help install")


In [ ]:
model, tokenizer = get_model_and_tokenizer(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [ ]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=100,
        max_steps=250,
        fp16=True,
        # push_to_hub=True
    )

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=2048
    )

Map:   0%|          | 0/8819 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss
10,1.384900
20,1.396800
30,1.396200
40,1.394300
50,1.390600
60,1.390100
70,1.386100
80,1.379600
90,1.380800
100,1.391100


TrainOutput(global_step=250, training_loss=1.3813264350891112, metrics={'train_runtime': 843.2025, 'train_samples_per_second': 18.975, 'train_steps_per_second': 0.296, 'total_flos': 1.1563576046014464e+16, 'train_loss': 1.3813264350891112, 'epoch': 1.8115942028985508})

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)

model_path = "output_model/checkpoint-250"

peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

model = peft_model.merge_and_unload()

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

In [ ]:
output_model_path = "/content/drive/MyDrive/DP/output_model"
model.save_pretrained(output_model_path)

KeyboardInterrupt: 

In [ ]:
from transformers import GenerationConfig
from time import perf_counter

def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

def generate_response(user_input):
  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=50,temperature=0.7,repetition_penalty=1.2,
      max_new_tokens=256,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  outputs = model.generate(**inputs, generation_config=generation_config)
  output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")
  return output



In [ ]:
generate_response(user_input='Swimmer')

Time taken for inference: 7.78 seconds


"<|im_start|>user\nSwimmer<|im_end|>\n<|im_start|>assistant:\nMy friend is a swimmer. He wants to sign up for competition in the future. But there's one thing that he has never done yet, and it scares him at the same time. He feels like he can't do it well enough. So I helped my friends get into practice sessions.<|im_end|>\n<|im_start|>assistant:\nThe summer before college started, my best friend signed up for swimming competitions. It made him nervous though because he didn't want to fail. The first day of training was awful, so my friend was not allowed to compete. However, his coach encouraged him to try again next year!<|im_end|>\n<|im_start|>assistant:\nYears later, my friend did make it to finals. After winning his race, I looked over at my friend's face with tears streaming down our faces. We both knew that he had conquered this fear by trying again. Now we wish all kids could enjoy sports without ever feeling scared!"

In [ ]:
generate_response("Monster Truck Rally")


Time taken for inference: 5.38 seconds


"<|im_start|>user\nMonster Truck Rally<|im_end|>\n<|im_start|>assistant:\nMy friend and I decided to go to a monster truck rally. We were excited about it, but we weren't sure what to expect. When we arrived at the rally, we saw all of these huge vehicles driving around. They looked like they could crush anything in their path.<|im_end|>\n</|im_start|>assistant:\nI don't have any photos or videos from my trip to the monster truck rally. It was just as exciting for me as watching movies!"

In [ ]:
generate_response("pop quiz")

Time taken for inference: 4.78 seconds


"<|im_start|>user\npop quiz<|im_end|>\n<|im_start|>assistant:\nThe teacher is giving a class quiz. The students are all nervous and unsure of what to do. They have been studying for weeks, but they still don't know how to answer the questions. One student starts to cry in front of everyone.<|im_end|>\n</|im_start|>assistant:\nOne of my friends was having trouble with a test at school. She didn't understand any of the questions. Her friend suggested she take a pop quiz online. It turned out that her friend had done it before!"

In [ ]:
generate_response("Charlie Horse")

Time taken for inference: 7.68 seconds


"<|im_start|>user\nCharlie Horse<|im_end|>\n<|im_start|>assistant:\nA few weeks ago, Charlie's horse died. He was devastated by the loss. His family and friends came to visit him in his home. They all cried together as they said goodbye to Charlie's beloved animal.<|im_end|>\n</|im_start|>assistant:\nThe man who had lost a pet recently decided that he wanted to adopt another one. After much research online, he found an adorable puppy for sale at a local shelter. The dog was exactly what he needed!"

In [ ]:
prediction_output_finetune_model = []

# Counter variable to track the number of iterations
counter = 0

# Iterate over each row in the test_df DataFrame
for index, row in test_df.iterrows():
    # Format the user input prompt
    user_input = f"user\n{row['storytitle']}\nassistant:"

    # Generate response using the generate_response function
    response = generate_response(user_input)

    # Append the generated response to the list
    prediction_output_finetune_model.append(response)

    # Increment the counter
    counter += 1

    # Check if the counter reaches 10, then break out of the loop
    if counter == 50:
        break

Time taken for inference: 2.71 seconds
Time taken for inference: 8.53 seconds
Time taken for inference: 3.39 seconds
Time taken for inference: 3.33 seconds
Time taken for inference: 2.7 seconds
Time taken for inference: 5.32 seconds
Time taken for inference: 8.82 seconds
Time taken for inference: 4.71 seconds
Time taken for inference: 9.31 seconds
Time taken for inference: 3.57 seconds
Time taken for inference: 2.02 seconds
Time taken for inference: 6.0 seconds
Time taken for inference: 4.51 seconds
Time taken for inference: 2.39 seconds
Time taken for inference: 3.49 seconds
Time taken for inference: 3.77 seconds
Time taken for inference: 4.01 seconds
Time taken for inference: 5.1 seconds
Time taken for inference: 1.95 seconds
Time taken for inference: 2.85 seconds
Time taken for inference: 1.34 seconds
Time taken for inference: 2.53 seconds
Time taken for inference: 4.86 seconds
Time taken for inference: 2.73 seconds
Time taken for inference: 1.52 seconds
Time taken for inference: 5.

In [ ]:
prediction_output_finetune_model

["<|im_start|>user\nuser\nPainful Collision\nassistant:<|im_end|>\n<|im_start|>assistant:I'm not aware of this particular car accident. However, I know there was a collision on the road. This person went to the hospital afterward and had surgery.<|im_end|>\n<|im_start|>assistant:I don't have first hand knowledge about these kinda situations. But I can tell you that accidents happen every day in our lives.<|im_end|>",
 "<|im_start|>user\nuser\nThe Blanket\nassistant:<|im_end|>\n<|im_start|>assistant:the blankets were made of soft and warm material. The first was a bright pink one that caught my eye on the shelf. I bought it hoping to make myself snug for bed in winter. Later, I learned that it's actually an animal skin and is too scratchy.<|im_end|>\n<|im_start|>assistant:I had a cat named Samson who like sleeping with me every night. He would roll over underneath me as if he was getting ready for his nap. His fur was so thick and soft that it felt comfortingly warm against his back.<|i

In [ ]:
def clean_generated_stories(stories):
    cleaned_stories = []
    for story in stories:
        # Remove "<|im_start|>" and "<|im_end|>"
        cleaned_story = story.replace("<|im_start|>", "")
        cleaned_story = cleaned_story.replace("<|im_end|>", "")

        # Remove "user:" and "assistant:"
        cleaned_story = cleaned_story.replace("user:", "")
        cleaned_story = cleaned_story.replace("assistant:", "")

        # Remove any remaining "\n" characters and extra whitespace
        cleaned_story = cleaned_story.replace("\n", " ")
        cleaned_story = cleaned_story.strip()

        cleaned_stories.append(cleaned_story)

    return cleaned_stories

prediction_output_finetune_model = clean_generated_stories(prediction_output_finetune_model)
print(prediction_output_finetune_model)


["useruserPainful CollisionI'm not aware of this particular car accident. However, I know there was a collision on the road. This person went to the hospital afterward and had surgery.I don't have first hand knowledge about these kinda situations. But I can tell you that accidents happen every day in our lives.", "useruserThe Blanketthe blankets were made of soft and warm material. The first was a bright pink one that caught my eye on the shelf. I bought it hoping to make myself snug for bed in winter. Later, I learned that it's actually an animal skin and is too scratchy.I had a cat named Samson who like sleeping with me every night. He would roll over underneath me as if he was getting ready for his nap. His fur was so thick and soft that it felt comfortingly warm against his back.Samson slept soundly next to me on many cold nights without ever complaining! Each time we wrapped him up in his soft blanket, he fell right into deep slumber.Whenever I got sick or tired at work, he came h

In [ ]:
prediction_output_finetune_model

["useruserPainful CollisionI'm not aware of this particular car accident. However, I know there was a collision on the road. This person went to the hospital afterward and had surgery.I don't have first hand knowledge about these kinda situations. But I can tell you that accidents happen every day in our lives.",
 "useruserThe Blanketthe blankets were made of soft and warm material. The first was a bright pink one that caught my eye on the shelf. I bought it hoping to make myself snug for bed in winter. Later, I learned that it's actually an animal skin and is too scratchy.I had a cat named Samson who like sleeping with me every night. He would roll over underneath me as if he was getting ready for his nap. His fur was so thick and soft that it felt comfortingly warm against his back.Samson slept soundly next to me on many cold nights without ever complaining! Each time we wrapped him up in his soft blanket, he fell right into deep slumber.Whenever I got sick or tired at work, he came 

# **Now we will work on the non-finetuned tinyllama model to comapre it's result with this model.**

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-scc4xgjo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-scc4xgjo
  Resolved https://github.com/huggingface/transformers.git to commit 73014b561d5f88d728e46a57d346f516fefe3f2d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9044069 sha256=5f031544596f65c94c81c93e75d6266b0664784ec5bc6f72809deb7932502c29
  Stored in directory: /tmp/pip-ephem-wheel-cache-gg5leyyq/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [ ]:
import pandas as pd
from transformers import pipeline
import torch
count = 0;
# Initialize the TinyLlama text generation pipeline
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", device_map="auto")

prediction_output_model = []
# Iterate through each row of the testing dataset
for _, row in test_df.iterrows():
    # Generate story using story title from the dataset
    messages = [{"role": "system", "content": "You help the user to generate a short story based on given sentence/pharse given by user"},
                                                  {"role": "user", "content": row["storytitle"]}]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    output = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    generated_story = output[0]["generated_text"]
    prediction_output_model.append(generated_story)
    print(row["storytitle"])
    print(generated_story)
    count += 1

    # Check if the counter reaches 10, then break out of the loop
    if count == 50:
        break

Painful Collision
<|system|>
You help the user to generate a short story based on given sentence/pharse given by user</s>
<|user|>
Painful Collision</s>
<|assistant|>
A painful collision is a situation where two or more objects or systems collide with each other. This can be caused by a variety of factors, such as a sudden impact or a misaligned system. In a short story, a painful collision could represent a traumatic event, such as a car accident or a workplace accident. The aftermath of the collision could create lasting emotional damage for the characters involved. The story could explore how the individuals affected by the collision try to process and cope with the trauma, and how the consequences of the collision impact their relationships and lives. The collision could also serve as a metaphor for larger societal issues, such as the impact of industrialization or the impact of globalization on local communities.
The Blanket
<|system|>
You help the user to generate a short story b

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Lost Hiker
<|system|>
You help the user to generate a short story based on given sentence/pharse given by user</s>
<|user|>
Lost Hiker</s>
<|assistant|>
Title: A Journey Beyond the Edge

Introduction:
The sun was setting behind the mountains as a lost hiker stumbled upon the path. He was lost and alone, his map and compass in his hands, but the path was unfamiliar. The wind whipped through his hair as he trudged through the dense forest, his heart racing with fear.

Act I:
The hiker trudged through the thick underbrush, his feet sinking into the mossy ground. The leaves rustled with the slightest breeze, and the rustling of the leaves echoed through the trees. The sound of his own breathing became louder as he fought to keep his footing.

Suddenly, he heard a rustling in the bushes. He spun around, his eyes wide with fear, and saw a small animal running towards him. The hiker froze, his heart pounding in his chest.

Act II:
The hiker's heart raced as he watched the small animal run tow

In [ ]:
prediction_output_model

['<|system|>\nYou help the user to generate a short story based on given sentence/pharse given by user</s>\n<|user|>\nPainful Collision</s>\n<|assistant|>\nA painful collision is a situation where two or more objects or systems collide with each other. This can be caused by a variety of factors, such as a sudden impact or a misaligned system. In a short story, a painful collision could represent a traumatic event, such as a car accident or a workplace accident. The aftermath of the collision could create lasting emotional damage for the characters involved. The story could explore how the individuals affected by the collision try to process and cope with the trauma, and how the consequences of the collision impact their relationships and lives. The collision could also serve as a metaphor for larger societal issues, such as the impact of industrialization or the impact of globalization on local communities.',
 '<|system|>\nYou help the user to generate a short story based on given sent

## Now final step is to compare accuracies of both the prediction done by both the model. ##

In [ ]:
testdata = Dataset.from_pandas(test_df)
testdata

Dataset({
    features: ['storytitle', 'sentence'],
    num_rows: 7900
})

In [ ]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=346d4a8a8b97fc8c048c985c57ea6af3db05aacd8cc206e26e58cd48cd1bcdf9
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1'])

# List to store ROUGE scores
rouge_scores = []

# Iterate through each pair of generated story and original sentence
for generated_story, original_sentence in zip(prediction_output_model, test_df["sentence"]):
    # Calculate the ROUGE score between the generated story and original sentence
    scores = scorer.score(generated_story, original_sentence)
    rouge_score = scores['rouge1'].fmeasure
    print(rouge_score)
    rouge_scores.append(rouge_score)

# Calculate the average ROUGE score
average_rouge_score = sum(rouge_scores) / len(rouge_scores)

print("Average ROUGE-1 Score for TinyLLama model:", average_rouge_score)





for generated_story, original_sentence in zip(prediction_output_finetune_model, test_df["sentence"]):
    # Calculate the ROUGE score between the generated story and original sentence
    scores = scorer.score(generated_story, original_sentence)
    rouge_score = scores['rouge1'].fmeasure
    print(rouge_score)
    rouge_scores.append(rouge_score)

# Calculate the average ROUGE score
average_rouge_score = sum(rouge_scores) / len(rouge_scores)

print("Average ROUGE-1 Score for fine-tuned TinyLLama model:", average_rouge_score)


0.1313131313131313
0.06584362139917697
0.09774436090225565
0.0885608856088561
0.22641509433962262
0.08403361344537814
0.10138248847926266
0.07547169811320754
0.052631578947368425
0.15384615384615385
0.20074349442379183
0.07999999999999999
0.15384615384615385
0.058333333333333334
0.13245033112582782
0.0813008130081301
0.15126050420168066
0.0398406374501992
0.06779661016949153
0.09558823529411765
0.10891089108910891
0.17692307692307693
0.1322314049586777
0.06542056074766354
0.1556420233463035
0.12167300380228138
0.10526315789473684
0.14634146341463414
0.13559322033898305
0.08450704225352113
0.07692307692307693
0.07441860465116279
0.140625
0.1450381679389313
0.10426540284360189
0.1285140562248996
0.23529411764705882
0.1376518218623482
0.06926406926406926
0.06060606060606061
0.11363636363636363
0.17435897435897435
0.1245136186770428
0.11538461538461539
0.08333333333333334
0.13245033112582782
0.07692307692307693
0.13793103448275865
0.10050251256281408
0.08547008547008547
Average ROUGE-1 Sco

In [ ]:
!pip install nltk gensim pot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 4.6 MB/s eta 0:00:00


In [ ]:
from time import time
start_nb = time()
from nltk.corpus import stopwords
from nltk import download
from gensim.models import Word2Vec

download('stopwords')
w2vmodel = Word2Vec(test_df["sentence"], vector_size=100, window=5, min_count=1, workers=4)

distances = []
for generated_story, original_sentence in zip(prediction_output_model, test_df["sentence"]):
    generated_story = generated_story.lower().split()
    original_sentence = original_sentence.lower().split()
    stop_words = stopwords.words('english')
    generated_story = [w for w in generated_story if w not in stop_words]
    original_sentence = [w for w in original_sentence if w not in stop_words]
    distance = w2vmodel.wv.wmdistance(generated_story, original_sentence)
    distances.append(distance)
    print("distance %0.4f", distance)

average_distance = sum(distances)/len(distances)
print("Average for Word mover's Score distance for TinyLLama model is:", average_distance)


distances = []
for generated_story, original_sentence in zip(prediction_output_finetune_model, test_df["sentence"]):
    generated_story = generated_story.lower().split()
    original_sentence = original_sentence.lower().split()
    stop_words = stopwords.words('english')
    generated_story = [w for w in generated_story if w not in stop_words]
    original_sentence = [w for w in original_sentence if w not in stop_words]
    distance = w2vmodel.wv.wmdistance(generated_story, original_sentence)
    distances.append(distance)
    print("distance %0.4f", distance)
average_distance = sum(distances)/len(distances)
print("Average for Word mover's Score distance for Fine-tuned TinyLLama model is:", average_distance)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf


distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
Average for Word mover's Score distance for TinyLLama model is: inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
d

distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
distance %0.4f inf
Average for Word mover's Score distance for Fine-tuned TinyLLama model is: inf


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# List to store BLEU scores
bleu_scores = []

# Iterate through each pair of generated output and original sentence
for generated_story, original_sentence in zip(prediction_output_model, test_df["sentence"]):
    # Tokenize the generated output and original sentence
    generated_story_tokens = generated_story.split()
    original_sentence_tokens = original_sentence.split()

    # Calculate BLEU score
    bleu_score = sentence_bleu([original_sentence_tokens], generated_story_tokens)
    bleu_scores.append(bleu_score)

# Calculate the average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU score:", avg_bleu_score)




Average BLEU score: 9.13313165123


In [ ]:
# List to store BLEU scores
bleu_scores = []

# Iterate through each pair of generated output and original sentence
for generated_story, original_sentence in zip(prediction_output_finetune_model, test_df["sentence"]):
    # Tokenize the generated output and original sentence
    generated_story_tokens = generated_story.split()
    original_sentence_tokens = original_sentence.split()

    # Calculate BLEU score
    bleu_score = sentence_bleu([original_sentence_tokens], generated_story_tokens)
    bleu_scores.append(bleu_score)
    print("BLEU score:", bleu_score)

# Calculate the average BLEU score
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU score:", avg_bleu_score)

Average BLEU score: 7.264832105465


In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyli

In [ ]:
import pandas as pd
test_df = pd.read_csv("testing_ROCStories.csv")

In [ ]:
avg_score1 = 0.810145
avg_score2 = 0.864154

In [ ]:
from bert_score import score
from bert_score import BERTScorer
refs = test_df["sentence"].tolist()[:50]  # Take the first 50 sentences from the reference data
cands = prediction_output_model  # Take the first 10 generated sentences as candidates

# Calculate BART score
scorer = BERTScorer(model_type="bert-base-uncased")
P, R, F1a = scorer.score(cands, refs, verbose=True)

# Print BERT score
# print("Bert score for TinyLLama model", F1.mean().item())

cands = prediction_output_finetune_model
P, R, F1b = scorer.score(cands, refs, verbose=True)

# Print BERT score
#print("Bert score for Finetuned TinyLLama model", F1.mean().item())
# print("Bert score for Finetuned TinyLLama model",)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

done in 7.41 seconds, 6.75 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.17 seconds, 42.91 sentences/sec


In [ ]:
print("Bert score for TinyLLama model",F1a.mean().item())

print("Bert score for Finetuned TinyLLama model",F1b.mean().item())


Bert score for TinyLLama model 0.81024516
Bert score for Finetuned TinyLLama model 0.8645201


In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

refs = test_df["sentence"].tolist()[:50]  # Take the first 50 sentences from the reference data
cands = prediction_output_model  # Take the first 10 generated sentences as candidates
# Calculate corpus BLEU score
corpus_bleu_score = corpus_bleu(refs, cands)

# Print corpus BLEU score
print("Corpus BLEU Score of the Tiny-LLama model:", corpus_bleu_score)



cands = prediction_output_finetune_model  # Take the first 10 generated sentences as candidates
# Calculate corpus BLEU score
corpus_bleu_score = corpus_bleu(refs, cands)

# Print corpus BLEU score
print("Corpus BLEU Score of fine-tuned model:", corpus_bleu_score)


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Corpus BLEU Score of the Tiny-LLama model: 7.209529799900637e-232
Corpus BLEU Score of fine-tuned model: 9.044893396637231e-232


In [ ]:
######### Backward BLue score

import nltk
from nltk.translate.bleu_score import corpus_bleu

refs = test_df["sentence"].tolist()[:50]  # Take the first 50 sentences from the reference data
cands = prediction_output_model  # Take the first 10 generated sentences as candidates
# Reverse the reference and candidate sentences
backward_refs = [ref[:-1] for ref in refs]
backward_cands = [cand[:-1] for cand in cands]

# Calculate backward BLEU score
backward_bleu_score = corpus_bleu(backward_refs, backward_cands)

# Print backward BLEU score
print("Backward BLEU Score of Tiny-LLama model:", backward_bleu_score)

cands = prediction_output_finetune_model
backward_cands = [cand[:-1] for cand in cands]

# Calculate backward BLEU score
backward_bleu_score = corpus_bleu(backward_refs, backward_cands)

# Print backward BLEU score
print("Backward BLEU Score of fine-tuned model:", backward_bleu_score)



Backward BLEU Score of Tiny-LLama model: 7.21121402561844e-232
Backward BLEU Score of fine-tuned model: 9.048513801532954e-232


In [ ]:
from sklearn.metrics import jaccard_score

def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

refs = test_df["sentence"].tolist()[:50]  # Take the first 50 sentences from the reference data
cands = prediction_output_model[:50]  # Take the first 50 generated sentences as candidates

# Convert reference and candidate sentences to sets
ref_sets = [set(ref) for ref in refs]
cand_sets = [set(cand) for cand in cands]

# Calculate Jaccard similarity score
jaccard_scores = [jaccard_similarity(ref_set, cand_set) for ref_set, cand_set in zip(ref_sets, cand_sets)]

# Print Jaccard similarity scores
print("Jaccard Similarity Scores (Model):", jaccard_scores)

# Calculate Jaccard similarity score for fine-tuned model
cands_finetune = prediction_output_finetune_model[:50]  # Take the first 50 generated sentences as candidates for fine-tuned model
cand_sets_finetune = [set(cand) for cand in cands_finetune]

# Calculate Jaccard similarity score
jaccard_scores_finetune = [jaccard_similarity(ref_set, cand_set) for ref_set, cand_set in zip(ref_sets, cand_sets_finetune)]

# Print Jaccard similarity scores
print("Jaccard Similarity Scores (Fine-tuned Model):", jaccard_scores_finetune)

# Calculate the average Jaccard similarity score for the model
avg_jaccard_score_model = sum(jaccard_scores) / len(jaccard_scores)

# Calculate the average Jaccard similarity score for the fine-tuned model
avg_jaccard_score_finetune = sum(jaccard_scores_finetune) / len(jaccard_scores_finetune)

# Print the average Jaccard similarity scores
print("Average Jaccard Similarity Score (Model):", avg_jaccard_score_model)
print("Average Jaccard Similarity Score (Fine-tuned Model):", avg_jaccard_score_finetune)



Jaccard Similarity Scores (Model): [0.5952380952380952, 0.48214285714285715, 0.54, 0.5416666666666666, 0.5957446808510638, 0.5306122448979592, 0.4807692307692308, 0.4909090909090909, 0.6041666666666666, 0.5581395348837209, 0.6382978723404256, 0.5306122448979592, 0.5652173913043478, 0.46153846153846156, 0.4807692307692308, 0.4716981132075472, 0.5909090909090909, 0.5185185185185185, 0.5918367346938775, 0.5192307692307693, 0.4423076923076923, 0.5581395348837209, 0.574468085106383, 0.6341463414634146, 0.52, 0.6829268292682927, 0.5555555555555556, 0.625, 0.43859649122807015, 0.4642857142857143, 0.5, 0.5652173913043478, 0.5094339622641509, 0.5490196078431373, 0.46296296296296297, 0.56, 0.7105263157894737, 0.5192307692307693, 0.5681818181818182, 0.5869565217391305, 0.5660377358490566, 0.5714285714285714, 0.5094339622641509, 0.5094339622641509, 0.4666666666666667, 0.6097560975609756, 0.45098039215686275, 0.627906976744186, 0.42857142857142855, 0.4339622641509434]
Jaccard Similarity Scores (Fin

In [ ]:
import subprocess

# Install nltk and rouge-score
subprocess.run(["pip", "install", "nltk", "rouge-score"])


CompletedProcess(args=['pip', 'install', 'nltk', 'rouge-score'], returncode=0)

In [ ]:
import os
import chardet

# Detect the encoding
locale_encoding = chardet.detect(b'hello')['encoding']

# Set the locale to UTF-8
os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'

# Install nltk and rouge-score
!pip install -q nltk rouge-score



In [ ]:
import subprocess

# Install rouge-score using subprocess
subprocess.run(["pip", "install", "-q", "rouge-score"])


CompletedProcess(args=['pip', 'install', '-q', 'rouge-score'], returncode=0)

In [ ]:
# Drop the 'text' column from the dataset
data = data.remove_columns('text')
data

In [ ]:
# Load the fine-tuned model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import pandas as pd
# model_name = model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

# Load the dataset
# data = pd.read_csv("your-dataset-path.csv")  # Replace "your-dataset-path.csv" with the path to your dataset CSV file

# Function to generate response
def generate_response(user_input):
    prompt = formatted_prompt(user_input)
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    generation_config = GenerationConfig(penalty_alpha=0.6, do_sample=True,
                                         top_k=5, temperature=0.5, repetition_penalty=1.2,
                                         max_new_tokens=2048, pad_token_id=tokenizer.eos_token_id)
    outputs = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output

# Calculate ROUGE score
def calculate_rouge_score(candidate, reference):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(candidate, reference)
    return scores['rougeL'].fmeasure

# Prepare references from the dataset
references = data['storytitle']

# Define a variable to keep track of the number of responses processed
num_responses = 0

# Initialize a variable to accumulate the total ROUGE score
total_rouge_score = 0

# Generate responses and evaluate
# Iterate over the dataset and access individual rows
for row in data:
    # Break the loop if the desired number of responses is reached
    if num_responses >= 10:
        break

    story_title = row['storytitle']
    sentence = row['sentence']

    # Generate response and evaluate
    generated_response = generate_response(story_title)

    # Ensure generated response and reference sentence are not empty
    if generated_response.strip() and sentence.strip():
        rouge_score = calculate_rouge_score(generated_response, sentence)
        print(f" ROUGE Score: {rouge_score}")
        total_rouge_score += rouge_score
        num_responses += 1

# Calculate the average ROUGE score
average_rouge_score = total_rouge_score / num_responses

print(f"Average ROUGE Score for {num_responses} responses: {average_rouge_score}")


 ROUGE Score: 0.1584158415841584
 ROUGE Score: 0.058823529411764705
 ROUGE Score: 0.1125
 ROUGE Score: 0.11392405063291139
 ROUGE Score: 0.1285714285714286
 ROUGE Score: 0.27826086956521734
 ROUGE Score: 0.09009009009009009
 ROUGE Score: 0.12871287128712872
 ROUGE Score: 0.17094017094017094
 ROUGE Score: 0.1267605633802817
Average ROUGE Score for 10 responses: 0.1366999415463152


In [ ]:
# !pip install bert_score
import subprocess

# Install rouge-score using subprocess
subprocess.run(["pip", "install", "-q", "bert_score"])


CompletedProcess(args=['pip', 'install', '-q', 'bert_score'], returncode=0)

In [ ]:
extracted_data = [
    {'storytitle': 'David Drops the Weight',
     'sentence': "David noticed he had put on a lot of weight recently. He examined his habits to try and figure out the reason. He realized he'd been eating too much fast food lately. He stopped going to burger places and started a vegetarian diet. After a few weeks, he started to feel much better."},
]
# Print the extracted data
print(extracted_data)

[{'storytitle': 'David Drops the Weight', 'sentence': "David noticed he had put on a lot of weight recently. He examined his habits to try and figure out the reason. He realized he'd been eating too much fast food lately. He stopped going to burger places and started a vegetarian diet. After a few weeks, he started to feel much better."}]


In [ ]:
from bert_score import BERTScorer

# Extract the story title and sentence from the provided data
story_title = extracted_data[0]['storytitle']
sentence = extracted_data[0]['sentence']

# Initialize BERTScorer
scorer = BERTScorer(model_type='bert-base-uncased')

# Calculate BERTScore
P, R, F1 = scorer.score([sentence], [story_title])

# Print the BERTScore
print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")


OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like bert-base-uncased is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
from bert_score import BERTScorer

# Load the fine-tuned model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import pandas as pd

# Load the dataset
# data = pd.read_csv("your-dataset-path.csv")  # Replace "your-dataset-path.csv" with the path to your dataset CSV file

# # Load the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("tokenizer")
# model = AutoModelForCausalLM.from_pretrained("model")

# Function to generate response
def generate_response(user_input):
  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=2048,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  outputs = model.generate(**inputs, generation_config=generation_config)
  output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")
  return output
# Initialize BERTScorer
scorer = BERTScorer(model_type="bert-base-uncased")

# Define a variable to keep track of the number of responses processed
num_responses = 0

# Initialize a variable to accumulate the total BERTScore
total_bert_score = 0

# Generate responses and evaluate
# Iterate over the dataset and access individual rows
for row in data:
    # Break the loop if the desired number of responses is reached
    if num_responses >= 20:
        break

    story_title = row['storytitle']
    sentence = row['sentence']

    # Generate response and evaluate
    print(story_title)
    generated_response = generate_response(user_input=story_title)
    print(generated_response)

    # Ensure generated response and reference sentence are not empty
    if generated_response.strip() and sentence.strip():
        # Calculate BERTScore
        P, R, F1 = scorer.score([generated_response], [sentence])
        bert_score = F1.mean().item()
        print(f" BERT Score: {bert_score}")
        total_bert_score += bert_score
        num_responses += 1

# Calculate the average BERTScore
average_bert_score = total_bert_score / num_responses

print(f"Average BERTScore for {num_responses} responses: {average_bert_score}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

David Drops the Weight
Time taken for inference: 5.59 seconds
<|im_start|>user
David Drops the Weight<|im_end|>
<|im_start|>assistant:
The man was in a weight loss slump. He had been trying for months to get back on track, but nothing seemed to be working. One day he decided that enough was enough and made an appointment with his doctor. The doctor gave him some exercises that would help him lose weight.<|im_end|>
<|im_start|>assistant:
After completing those exercises, David lost 10 pounds! He felt great and continued to exercise regularly after that. His wife noticed how much better he looked too!<|im_end|>
<|im_start|>assistant:
Now David is happier than ever and has kept himself healthy by sticking to his fitness routine.<|im_end|>
 BERT Score: 0.48090633749961853
Frustration
Time taken for inference: 9.11 seconds
<|im_start|>user
Frustration<|im_end|>
<|im_start|>assistant:
I was frustrated with my roommate. He always made a mess in the kitchen and didn't clean up after himself. I

In [ ]:
from bert_score import BERTScorer

# Load the fine-tuned model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import pandas as pd

# Load the dataset
# data = pd.read_csv("your-dataset-path.csv")  # Replace "your-dataset-path.csv" with the path to your dataset CSV file

# # Load the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("tokenizer")
# model = AutoModelForCausalLM.from_pretrained("model")

# Function to generate response
import re

def generate_response(user_input):
    prompt = formatted_prompt(user_input)

    inputs = tokenizer([prompt], return_tensors="pt")
    generation_config = GenerationConfig(penalty_alpha=0.6, do_sample=True,
                                          top_k=5, temperature=0.6, repetition_penalty=1.2,
                                          max_new_tokens=2048, pad_token_id=tokenizer.eos_token_id
                                          )
    start_time = perf_counter()

    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    outputs = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Splitting the output into sentences
    sentences = re.split(r'(?<=[.!?]) +', output)

    # Concatenating only the first two sentences
    final_output = ' '.join(sentences[:6])

    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time, 2)} seconds")
    return final_output

# Initialize BERTScorer
scorer = BERTScorer(model_type="bert-base-uncased")

# Define a variable to keep track of the number of responses processed
num_responses = 0

# Initialize a variable to accumulate the total BERTScore
total_bert_score = 0

# Generate responses and evaluate
# Iterate over the dataset and access individual rows
for row in data:
    # Break the loop if the desired number of responses is reached
    if num_responses >= 20:
        break

    story_title = row['storytitle']
    sentence = row['sentence']

    # Generate response and evaluate
    print(story_title)
    generated_response = generate_response(user_input=story_title)
    print(generated_response)

    # Ensure generated response and reference sentence are not empty
    if generated_response.strip() and sentence.strip():
        # Calculate BERTScore
        P, R, F1 = scorer.score([generated_response], [sentence])
        bert_score = F1.mean().item()
        print(f" BERT Score: {bert_score}")
        total_bert_score += bert_score
        num_responses += 1

# Calculate the average BERTScore
average_bert_score = total_bert_score / num_responses

print(f"Average BERTScore for {num_responses} responses: {average_bert_score}")


David Drops the Weight
Time taken for inference: 7.9 seconds
<|im_start|>user
David Drops the Weight<|im_end|>
<|im_start|>assistant:
Sarah's husband David was overweight. He had tried to lose weight for years but never really made any progress. Sarah decided she would help him. She started by giving him a lot of love and support. After a while, he lost some weight. Now he is happy with his new healthy lifestyle.<|im_end|>
<|im_start|>assistant:
Sarah was proud that her husband had taken control of his life again.
 BERT Score: 0.48168909549713135
Frustration
Time taken for inference: 5.0 seconds
<|im_start|>user
Frustration<|im_end|>
<|im_start|>assistant:
I'm frustrated because I can't find my car keys. The last time it happened, they were in the glove box and all I had to do was open that door. This time, I didn't have that option and now I'm stuck outside with no car. Frustration is setting in!<|im_end|>
<|im_start|>assistant:
The assistant went back inside their house to try and ge

In [ ]:
print(f"Average BERTScore for {num_responses} responses: {average_bert_score}")

Average BERTScore for 10 responses: 0.8640954200029183


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

# List to store generated responses and reference sentences
generated_responses = []
reference_sentences = []

# Initialize variables to keep track of the number of responses processed and total BLEU score
num_responses = 0
total_bleu_score = 0

# Iterate over the dataset and access individual rows
for row in data:
    # Break the loop if the desired number of responses is reached
    if num_responses >= 10:
        break

    story_title = row['storytitle']
    sentence = row['sentence']

    # Generate response and add to generated_responses list
    generated_response = generate_response(story_title)
    generated_responses.append(generated_response.split())

    # Add reference sentence to reference_sentences list
    reference_sentences.append([sentence.split()])

    num_responses += 1

    # Calculate BLEU score for the current response
    bleu_score = corpus_bleu(reference_sentences, generated_responses)
    total_bleu_score += bleu_score

# Calculate average BLEU score
average_bleu_score = total_bleu_score / num_responses if num_responses > 0 else 0

print("Average Corpus BLEU Score for responses:", average_bleu_score)


Time taken for inference: 7.55 seconds
Time taken for inference: 4.82 seconds
Time taken for inference: 2.54 seconds
Time taken for inference: 8.72 seconds
Time taken for inference: 12.01 seconds
Time taken for inference: 2.86 seconds
Time taken for inference: 10.17 seconds
Time taken for inference: 6.47 seconds
Time taken for inference: 3.56 seconds
Time taken for inference: 9.78 seconds
Average Corpus BLEU Score for responses: 5.904466070852662e-79


In [ ]:

# !pip install bert_score
import subprocess

# Install rouge-score using subprocess
subprocess.run(["pip", "install", "-q", "rouge-score"])


CompletedProcess(args=['pip', 'install', '-q', 'rouge-score'], returncode=0)

In [ ]:
def generate_response(user_input):
    prompt = formatted_prompt(user_input)

    inputs = tokenizer([prompt], return_tensors="pt")
    generation_config = GenerationConfig(penalty_alpha=0.6, do_sample=True,
                                          top_k=5, temperature=0.6, repetition_penalty=1.2,
                                          max_new_tokens=2048, pad_token_id=tokenizer.eos_token_id
                                          )
    start_time = perf_counter()

    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    outputs = model.generate(**inputs, generation_config=generation_config)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Splitting the output into sentences
    sentences = re.split(r'(?<=[.!?]) +', output)

    # Concatenating only the first two sentences
    final_output = ' '.join(sentences[:6])

    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time, 2)} seconds")
    return final_output

In [ ]:
# !pip install transformers
import subprocess

# Install rouge-score using subprocess
subprocess.run(["pip", "install", "-q", "transformers"])



CompletedProcess(args=['pip', 'install', '-q', 'transformers'], returncode=0)

In [ ]:
from transformers import pipeline

# Load the BLEURT model
# bleurt_pipeline = pipeline("text-scoring", model="bleurt-base-uncased")
bleurt_pipeline = pipeline("text-scoring", model="google/bleurt-base-uncased-finetuned-yelp")


# Function to calculate BLEURT score
def calculate_bleurt_score(candidate, reference):
    score = bleurt_pipeline(candidate, reference)[0]['score']
    return score

# Initialize variables to keep track of the number of responses processed and total BLEURT score
num_responses = 0
total_bleurt_score = 0

# Iterate over the dataset and access individual rows
for row in data:
    # Break the loop if the desired number of responses is reached
    if num_responses >= 20:
        break

    story_title = row['storytitle']
    sentence = row['sentence']

    # Generate response and evaluate
    generated_response = generate_response(story_title)

    # Calculate BLEURT score for the current response
    bleurt_score = calculate_bleurt_score(generated_response, sentence)
    total_bleurt_score += bleurt_score

    num_responses += 1

# Calculate average BLEURT score
average_bleurt_score = total_bleurt_score / num_responses if num_responses > 0 else 0

print("Average BLEURT Score for responses:", average_bleurt_score)


OSError: google/bleurt-base-uncased-finetuned-yelp is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# Function to calculate Backward BLEU score
def calculate_backward_bleu_score(candidate, reference):
    # Reverse the order of tokens in both candidate and reference
    candidate_reversed = candidate[::-1]
    reference_reversed = reference[::-1]

    # Calculate BLEU score using the reversed sequences
    bleu_score = sentence_bleu([reference_reversed], candidate_reversed)
    return bleu_score

# Initialize variables to keep track of the number of responses processed and total Backward BLEU score
num_responses = 0
total_backward_bleu_score = 0

# Iterate over the dataset and access individual rows
for row in data:
    # Break the loop if the desired number of responses is reached
    if num_responses >= 10:
        break

    story_title = row['storytitle']
    sentence = row['sentence']

    # Generate response and evaluate
    generated_response = generate_response(story_title)

    # Calculate Backward BLEU score for the current response
    backward_bleu_score = calculate_backward_bleu_score(generated_response.split(), sentence.split())
    total_backward_bleu_score += backward_bleu_score

    num_responses += 1

# Calculate average Backward BLEU score
average_backward_bleu_score = total_backward_bleu_score / num_responses if num_responses > 0 else 0

print("Average Backward BLEU Score for 1000 responses:", average_backward_bleu_score)


Time taken for inference: 4.76 seconds


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Time taken for inference: 4.09 seconds
Time taken for inference: 4.15 seconds
Time taken for inference: 6.63 seconds
Time taken for inference: 7.0 seconds
Time taken for inference: 3.5 seconds
Time taken for inference: 2.38 seconds
Time taken for inference: 5.0 seconds
Time taken for inference: 5.95 seconds


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Time taken for inference: 6.48 seconds
Average Backward BLEU Score for 1000 responses: 0.006837203339116285


In [ ]:
!pip install requests

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
test_df = pd.read_csv("testing_ROCStories.csv")
test_df

,storytitle,sentence
0,Painful Collision,Kyle traveled to the beach. He took his surfbo...
1,The Blanket,"Franny was always cold. She bought a big, fluf..."
2,Pop Culture,Carrie's man was cheating on her and she was v...
3,shoes,Joana loves buying shoes but not wearing it. O...
4,Terry's love,Terry's grandparent was very sick in the hospi...
...,...,...
7895,School Lunch,Amanda had just started kindergarten. Her moth...
7896,Stalking Ex,My ex husband has been calling me every day. I...
7897,Bitter Beer,Ken never liked drinking beer. After a while h...
7898,An Unwelcome Encounter,Dan opened the lattice door to get access unde...


In [ ]:
import pandas as pd
def ms_jaccard_similarity(list1, list2):
    """
    Calculate the MS-Jaccard similarity between two lists of strings.

    Args:
    - list1: A list of strings.
    - list2: A list of strings.

    Returns:
    - MS-Jaccard similarity score.
    """
    # Convert lists to dictionaries with frequency counts
    set_A = {elem: list1.count(elem) for elem in set(list1)}
    set_B = {elem: list2.count(elem) for elem in set(list2)}

    intersection = sum(min(set_A.get(elem, 0), set_B.get(elem, 0)) for elem in set_A.keys() | set_B.keys())
    union = sum(max(set_A.get(elem, 0), set_B.get(elem, 0)) for elem in set_A.keys() | set_B.keys())

    if union == 0:  # Avoid division by zero
        return 0

    return intersection / union

# Example usage:
list1 = ["apple", "banana", "banana", "orange", "orange", "orange"]
list2 = ["banana", "banana", "grape", "grape", "kiwi", "kiwi", "kiwi"]
test_df = pd.read_csv("testing_ROCStories.csv")
tets_df_list = test_df["sentence"].tolist()[:50]

score = ms_jaccard_similarity(prediction_output_model, tets_df_list)
print("MS-Jaccard similarity score:", score)

MS-Jaccard similarity score: 0.0
